### 사전준비

### 환경설정


In [1]:
# 한글 깨짐을 방지하는 코드
import matplotlib.pyplot as plt

# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
print(f)

# 확인 이후
plt.rc('font', family='Malgun Gothic')

['cmtt10', 'DejaVu Sans', 'STIXGeneral', 'DejaVu Sans Mono', 'STIXSizeFourSym', 'STIXSizeFiveSym', 'STIXSizeTwoSym', 'DejaVu Sans Mono', 'STIXGeneral', 'DejaVu Serif', 'STIXNonUnicode', 'STIXSizeOneSym', 'DejaVu Sans Mono', 'cmr10', 'STIXNonUnicode', 'STIXSizeTwoSym', 'STIXSizeOneSym', 'DejaVu Serif Display', 'DejaVu Sans', 'STIXGeneral', 'STIXGeneral', 'DejaVu Sans', 'cmmi10', 'STIXSizeThreeSym', 'cmb10', 'DejaVu Serif', 'STIXSizeThreeSym', 'DejaVu Sans', 'cmss10', 'DejaVu Sans Mono', 'DejaVu Serif', 'DejaVu Serif', 'cmex10', 'cmsy10', 'STIXNonUnicode', 'STIXSizeFourSym', 'STIXNonUnicode', 'DejaVu Sans Display', 'Tahoma', 'Gadugi', 'Bradley Hand ITC', 'SimSun-ExtB', 'Berlin Sans FB Demi', 'OCR A Extended', 'Calibri', 'Microsoft New Tai Lue', 'Lucida Sans', 'Segoe UI', 'Lucida Sans Unicode', 'HYHeadLine-Medium', 'Broadway', 'Sylfaen', 'Papyrus', 'Rockwell Condensed', 'Trebuchet MS', 'Cambria', 'Pyunji R', 'Bell MT', 'Candara', 'Cambria', 'Courier New', 'Elephant', 'New Gulim', 'Century

In [2]:
import pandas as pd
import numpy as np
import re
import io
import os
from tqdm.notebook import tqdm
tqdm.pandas()
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score

# GPU 메모리 및 최대 시퀀스 길이에 따른 배치 수 설정이 필요
# 시퀀스 길이가 짧다면 32 이상의 배치 사이즈를 사용하는 것이 가능
batch_size = 32

data_path ='./판결문'
# data_path ='./archive/라벨링데이터/판결문'

# 분류 라벨
labels_ids = {'01_주장':0, '02_사실':1, '03_판단':2, '04_결론':3}

# How many labels are we using in training.
# This is used to decide size of classification head.
n_labels = len(labels_ids)

### 학습데이터 전처리

In [3]:
import json
def is_json_key_present(json, key):
    try:
        buf = json[key]
    except KeyError:
        return False
    return True

class JsonDataset(Dataset):
 

  def __init__(self, path):

    # Check if path exists.
    if not os.path.isdir(path):
      # Raise error if path is invalid.
      raise ValueError('Invalid `path` variable! Needs to be a directory')
    self.texts = []
    self.labels = []

    
    for label in ['01.민사', '02.형사', '03.행정']:
      sentiment_path = os.path.join(path, label)

      # Get all files from path.
      for (root, directories, files) in os.walk(sentiment_path):
      # Go through each file and read its content.
        for file_name in tqdm(files, desc=f'{root} files'):
          file_path = os.path.join(root, file_name)          
          with open(file_path, "r") as json_file:
            json_data  =  json.load(json_file)        
            if(is_json_key_present(json_data,"assrs") and is_json_key_present(json_data['assrs'],"acusrAssrs")) :             
              for data in json_data['assrs']['acusrAssrs'] :
                self.texts.append(fix_text(data))
                self.labels.append('01_주장')

            if(is_json_key_present(json_data,"assrs") and is_json_key_present(json_data['assrs'],"dedatAssrs")) :
              for data in json_data['assrs']['dedatAssrs'] :
                self.texts.append(fix_text(data))
                self.labels.append('01_주장')

            if(is_json_key_present(json_data,"facts") and is_json_key_present(json_data['facts'],"bsisFacts")) :
              for data in json_data['facts']['bsisFacts'] :
                self.texts.append(fix_text(data))
                self.labels.append('02_사실')

        
            if(is_json_key_present(json_data,"dcss") and is_json_key_present(json_data['dcss'],"courtDcss")) :
              for data in json_data['dcss']['courtDcss'] :
                self.texts.append(fix_text(data))
                self.labels.append('03_판단')
      
            if(is_json_key_present(json_data,"close") and is_json_key_present(json_data['close'],"cnclsns")) :
              for data in json_data['close']['cnclsns'] :

                self.texts.append(fix_text(data))
                self.labels.append('04_결론')
    # Number of exmaples.
    self.n_examples = len(self.labels)
    return
  
  def __len__(self):
    return self.n_examples

  def __getitem__(self, item):
    return {'text':self.texts[item],
            'label':self.labels[item]}

### 모델 초기화

## 학습데이터 LOAD


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# JSON Data 
dataset = JsonDataset(path=data_path)
df = pd.DataFrame.from_records(dataset)
df.groupby('label').size()

df = df.sample(frac=0.05, random_state=123)

df1 = df[df['label'].str.startswith('01')]
df2 = df[df['label'].str.startswith('02')]
df3 = df[df['label'].str.startswith('03')]
df4 = df[df['label'].str.startswith('04')]        

# 전체 데이터에서 train, test 분리 (9 : 1)
df1_train, df1_test = train_test_split(df1, test_size=0.1, random_state=123)
df2_train, df2_test = train_test_split(df2, test_size=0.1, random_state=123)
df3_train, df3_test = train_test_split(df3, test_size=0.1, random_state=123)
df4_train, df4_test = train_test_split(df4, test_size=0.1, random_state=123)

df_train = pd.concat([df1_train,df2_train,df3_train,df4_train])
df_test = pd.concat([df1_test,df2_test,df3_test,df4_test])

c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. P

./판결문\01.민사 files: 0it [00:00, ?it/s]

c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. P

./판결문\01.민사\1981~2016 files:   0%|          | 0/3257 [00:00<?, ?it/s]

c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. P

...

./판결문\03.행정\2019 files:   0%|          | 0/75 [00:00<?, ?it/s]

c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. P

./판결문\03.행정\2020 files:   0%|          | 0/42 [00:00<?, ?it/s]

c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
c:\Users\ben81\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. P

./판결문\03.행정\2021 files:   0%|          | 0/108 [00:00<?, ?it/s]

In [5]:
#전체데이터 확인
df.groupby('label').size()

label
01_주장    1828
02_사실    3203
03_판단    5817
04_결론    1179
dtype: int64

In [6]:
# 한국어 형태소 분석기 Okt를 사용하기 위해 import
from konlpy.tag import Okt
# Okt의 객체 생성
okt = Okt()

# 한국어에서 의미가 거의 없는 단어들을 불용어 처리하기 위해 집합을 만듦
stop_words = set(['은', '는', '이', '가', '아', '하', '들', '것', '의', '있', '되', '수', '보', '주', '등', '한', '에'])

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
# 전처리 함수
def preprocessing(okt, review):
  # 한글이 아닌 단어는 지우고 " "으로 대체
  review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]", " ", review)
  # 형태소 분석을 하며 stemming 기법을 사용
  token_li = okt.morphs(review_text, stem=True)
  # 이전 stop_words에 포함되지 않는 단어만 clean_review에 들어갈 수 있게 함
  clean_review = [token for token in token_li if not token in stop_words]
  # 단어들을 ' '기준으로 이어 붙혀서 하나의 string 타입의 변수를 생성함
  clean_review = ' '.join(clean_review)

  # 전처리 완료된 문장을 return
  return clean_review.strip()

In [ ]:
# preprocessing 함수를 적용
df_train["prepro_data"] = df_train["text"].progress_apply(lambda x: preprocessing(okt, x))
df_test["prepro_data"] = df_test["text"].progress_apply(lambda x: preprocessing(okt, x))

  0%|          | 0/10823 [00:00<?, ?it/s]

  0%|          | 0/1204 [00:00<?, ?it/s]

In [ ]:
# 전처리 과정에서 모든 문장이 삭제될 가능성이 있으므로, 해당 데이터 제거

print("공백 제거 전 훈련 데이터 개수: ", len(df_train))
train_not_empty_value = df_train["prepro_data"] != ""
df_train = df_train[train_not_empty_value]
print("공백 제거 후 훈련 데이터 개수: ", len(df_train))

print("공백 제거 전 태스트 데이터 개수: ", len(df_test))
test_not_empty_value = df_test["prepro_data"] != ""
df_test = df_test[test_not_empty_value]
print("공백 제거 후 테스트 데이터 개수: ", len(df_test))

공백 제거 전 훈련 데이터 개수:  10823
공백 제거 후 훈련 데이터 개수:  10821
공백 제거 전 태스트 데이터 개수:  1204
공백 제거 후 테스트 데이터 개수:  1204


In [ ]:
# 순서 변경
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

# 훈련 데이터
x_train = df_train["prepro_data"].copy()
y_train = df_train["label"].copy()

# 테스트 데이터
x_test = df_test["prepro_data"].copy()
y_test = df_test["label"].copy()

In [ ]:
# tfidf embedding
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)
tfidf_embeddings = vectorizer.transform(text_data)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TF-IDF vectorizer object
vectorizer = TfidfVectorizer()

# Fit the vectorizer on your text data
vectorizer.fit(text_data)

# Transform the text data into TF-IDF embeddings
tfidf_embeddings = vectorizer.transform(text_data)

In [ ]:
# modeling

In [ ]:
# confusion matric 출력

In [ ]:
# 결과 리포트 출력
